In [ ]:
from tensorflow.keras import Sequential, utils
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# add channel to images (required by tensorflow)
img_shape=(28, 28, 1)
X_train = X_train.reshape(X_train.shape[0], *img_shape)
X_test = X_test.reshape(X_test.shape[0], *img_shape)

# convert images to floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# scale data between [0,1]
X_train /= 255
X_test /= 255

# convert labels to categorical
num_classes = 10
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=img_shape))

model.add(Conv2D(64, kernel_size=3, strides=2, activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

# predict class
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

In [ ]:
# remove previous logs
%rm -r logs/ 2> /dev/null

# create callback to store history
tb = TensorBoard('logs')
tb.set_model(model)

In [ ]:
epochs = 10
batch_size = 128
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[tb])